In [1]:
import csv
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
#Twitter API credentials
consumer_key = "S4rBwYqakKu2yfRRu4ehmCeqSA"
consumer_secret = "0BBlr7xtHRHvtgF8YUNI93hkfVccxS2BYqD0VaUip16anebDpYA"
access_key = "340236864-Uyejm2wMkKvgK7MbhJOzG0mBjAiZV5dMDUvZNkqtA"
access_secret = "zoaGsEg9nkrH0fIJUUOv3PfI5lg3IqhbnKJOWQ3t2UV9vA"

In [3]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

api = tweepy.API(auth)

In [4]:
#columns of the csv file
COLS = ['Tweet','Date Created','Subjectivity','Polarity','Sentiment']
df = pd.DataFrame(columns=COLS) 

In [5]:
#Define the search terms and the date_since date as variables
account_name = "spectatorindex"
key_words_list = ["coronavirus","corona","covid","covid-19"]
#date_since = "2019-01-01"
item_count = 300

In [6]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
 
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)
 
 #mrhod clean_tweets()
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
 
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
 
 
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
 
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
 
    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)

In [7]:
# Collect tweets
tweets = tweepy.Cursor(api.user_timeline, screen_name = account_name).items(item_count)

In [8]:
# Iterate and print tweets
for tweet in tweets:
    original_tweet=tweet.text
    time_created= tweet.created_at
    #call clean_tweet method for extra preprocessing
    filtered_tweet=clean_tweets(tweet.text) 
    #pass textBlob method for sentiment calculations
    blob = TextBlob(filtered_tweet)
    Sentiment = blob.sentiment 
    #seperate polarity and subjectivity in to two variables
    polarity = Sentiment.polarity
    subjectivity = Sentiment.subjectivity    
    #set sentiment 
    if polarity > 0: 
        sent='positive'
    elif polarity == 0: 
        sent='neutral'
    else: 
        sent='negative'
    df = df.append({'Tweet': '%s' %original_tweet,'Date Created': '%s' %time_created ,'Subjectivity': '%s'%subjectivity, 'Polarity': '%s' %polarity,'Sentiment':'%s'%sent}, ignore_index=True)
df.head()

,Tweet,Date Created,Subjectivity,Polarity,Sentiment
0,"Coronavirus in Germany\n\nCases: 3,675\n\nReco...",2020-03-14 11:30:00,0.0,0.0,neutral
1,"Coronavirus in Spain\n\nCases: 5,232\n\nRecove...",2020-03-14 11:00:01,0.0,0.0,neutral
2,"Coronavirus in South Korea\n\nCases: 8,086\n\n...",2020-03-14 10:53:55,0.0,0.0,neutral
3,"Coronavirus in Italy\n\nCases: 17,660\n\nRecov...",2020-03-14 10:45:00,0.0,0.0,neutral
4,"Coronavirus in China\n\nCases: 80,824\n\nRecov...",2020-03-14 10:38:40,0.0,0.0,neutral


In [9]:
#filter dataframe based on key words
df2 = df[df['Tweet'].str.contains('|'.join(key_words_list))]
df2.head()

,Tweet,Date Created,Subjectivity,Polarity,Sentiment
5,JUST IN: Fourth coronavirus death in Belgium,2020-03-14 10:35:48,0.0,0.0,neutral
6,"BREAKING: 97 new coronavirus deaths in Iran, r...",2020-03-14 10:31:10,0.45454545454545453,0.13636363636363635,positive
7,JUST IN: Saudi Arabia suspends international f...,2020-03-14 03:55:44,0.0,0.0,neutral
8,JUST IN: 13 new coronavirus deaths in China,2020-03-14 02:26:14,0.45454545454545453,0.13636363636363635,positive
9,JUST IN: Brazil's ambassador to the US tests p...,2020-03-14 02:14:00,0.5454545454545454,0.22727272727272727,positive


In [10]:
df2.to_csv('corona_sentiment-2020-03-14-rev4.csv', encoding='utf-8', index=False)